## Focus on geolocations in specific cities

In [2]:
%load_ext autoreload
%autoreload 2
%cd D:\mobi-seg-net

D:\mobi-seg-net


In [3]:
import pandas as pd
import numpy as np
import os
os.environ['JAVA_HOME'] = "C:/Java/jdk-1.8"
from tqdm import tqdm
import sqlalchemy
from lib import workers as workers
import matplotlib.pyplot as plt

In [4]:
def within_box(lat, lng):
    if (lat >= workers.stockholm_box[1]) & (lat <= workers.stockholm_box[3]):
        if (lng >= workers.stockholm_box[0]) & (lng <= workers.stockholm_box[2]):
            return 1
    return 0

In [8]:
df_stops_list = []
for i in tqdm(range(0, 50), desc="Filtering data"):
    df_stops = pd.read_parquet(f"dbs/stops_pr/stops_pr_{i}.parquet", columns=['device_aid', 'h3_id', 'home', 'kind', 'latitude', 'longitude'])
    df_stops = df_stops[df_stops['home']!=1]
    df_stops.loc[:, 'Stockholm'] = df_stops.apply(lambda row: within_box(row['latitude'], row['longitude']), axis=1)
    df_stops = df_stops.loc[df_stops['Stockholm'] == 1, :].drop(columns=['Stockholm', 'latitude', 'longitude'])
    df_stops_list.append(df_stops)

Filtering data: 100%|██████████| 50/50 [11:28<00:00, 13.77s/it]


In [9]:
df_stops = pd.concat(df_stops_list)
df_stops.drop(columns=['home'], inplace=True)
print(len(df_stops), df_stops.device_aid.nunique())

24609483 941746


In [13]:
df_stops.to_parquet('dbs/cities/stockholm.parquet', index=False)